In [2]:
# Import Library
import numpy as np
import scipy.integrate as spi
import scipy.interpolate as spn
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, HTML
import voila

In [13]:
def run_sirv(days, N, I0, R0, infectious_period, r0, vacp,vacf):
    gamma = 1/infectious_period
    beta = gamma*r0

#     I0 = 1
#     R0 = 0
    S0 = N-I0-R0

    t = np.linspace(1,days, days)
    # Initial coditions vector
    y0 = S0,I0,R0 
    
    # p
    pStart = vacp[0]
    pEnd = vacp[1]
    pFrac = vacf
    
    p = np.zeros(days)
    a = pFrac/(pEnd-pStart)
    x = np.linspace(0,(pEnd-pStart),(pEnd-pStart)-1)
    p[0:pStart-1] = 0
    p[pStart:pEnd-1] = a*x
    p[pEnd-1:] = pFrac

    interp = spn.interp1d(t,p)

    # Model Deinition
    def sirv(t,y,N,beta, gamma):
        S,I,R = y
        p = interp(t)
        dSdt = -beta*(1-p)*S*I/N
        dIdt = beta*(1-p)*S*I/N - gamma*I
        dRdt = gamma*I
        return dSdt, dIdt, dRdt

    
    sol = spi.solve_ivp(sirv,(1,days),y0, args=(N,beta,gamma),t_eval=t, method='RK45')
    
    I = sol.y[1]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t,y=I, mode="lines", name="Active Cases", line={'dash': 'solid', 'color': 'red'}))
    #X-Axis
    fig.update_xaxes(title_text='Days')
    
    #Y-axis
    fig.update_yaxes(title_text='Count')
    
    # Layout
    fig.update_layout(title={"text": "SIR Model with Vaccination","x": 0.5,"y": .95,"xanchor": "center","yanchor": "bottom"}, #Title
                      width=740, height=420, #Figure size
                      margin=dict(r=20, b=10, l=10, t=30),
                      template = 'plotly_white', #Template {seaborn, plotly_white, dark}
                      showlegend = True,
                       annotations=[
                        dict(
                            x=0.97,
                            y=0.03,
                            xref="paper",
                            yref="paper",
                            text="Kaustuv",
                            ax=0,
                            ay=0
                            )
                        ]
                      )     
    
    # Annotations
    max_infected = max(I)
    max_pos = np.argmax(I)
    day_max_infected = t[max_pos]
    txt = "Max Active Cases: {a:.0f} on Day: {b:.0f}"
    max_text = txt.format(a=max_infected, b=day_max_infected)
    fig.add_annotation(text=max_text,
                       x=day_max_infected ,y=max_infected,
                       xref='x', yref='y',
#                       ax=0, ay=0,
                       showarrow=True)
    
    fig.add_vrect(
    x0=pStart, x1=pEnd,
    fillcolor="Green", opacity=0.25,
    layer="above", line_width=1)
        

    
    fig.show()


In [15]:
style = {'description_width': 'initial'}
days = widgets.IntSlider(value=300, min=100, max=10000, step=100, description='Simulation Days:', style=style, layout=widgets.Layout(min_width='600px'))
max_d = days.value
N = widgets.IntSlider(value=1.387e9, min=0, max=1.387e9, step=1e6, description='Population:', style=style, layout=widgets.Layout(min_width='600px'))
I0 = widgets.IntSlider(value=1, min=1, max=1e9, step=1000, description='Active Cases:', style=style, layout=widgets.Layout(min_width='600px'))
R0 = widgets.IntSlider(value=0, min=0, max=1e9, step=1000, description='Recovered:', style=style, layout=widgets.Layout(min_width='600px'))

infectious_period = widgets.FloatSlider(value=10, min=1, max=20, step=0.1, description='Infectious Period:', style=style, layout=widgets.Layout(min_width='600px'))
r0 = widgets.FloatSlider(value=2.5, min=0, max=10, step=0.1, description='R0:', style=style, layout=widgets.Layout(min_width='600px'))

vacp = widgets.IntRangeSlider(value=[30, 90], min=0, max=max_d, step=1, description='Vaccination Period:', style=style, layout=widgets.Layout(min_width='600px'))
vacf = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.1, description='Vaccination Fraction:', style=style, layout=widgets.Layout(min_width='600px'))

f = widgets.interact(run_sirv, days=days, N=N, I0=I0, R0=R0, infectious_period=infectious_period, r0=r0, vacp=vacp,vacf=vacf)


interactive(children=(IntSlider(value=300, description='Simulation Days:', layout=Layout(min_width='600px'), m…